In [1]:
import numpy as np 
import rospy 
import sys
sys.path.append("..")
from kinematics.utils.util_ik import make_ik_input
from kinematics.class_structure import CHAIN
from kinematics.utils.util_ik import add_joints, find_route
print("Done.")

Done.


In [2]:
class RobotClass(object):
    def __init__(self, file_name, base_offset):
        super(RobotClass,self).__init__() 
        self.chain = CHAIN(file_name=file_name, base_offset=base_offset, verbose=True)
        self.chain.add_joi_to_robot()
        self.chain.add_link_to_robot()
        self.chain.fk_chain(1)
    
    def solve_ik(self,  target_name      = ['wrist_3_joint'],
                        target_position  = [[0,0,0]],
                        target_rotation  = [[-1.57, 0, -1.57]],
                        solve_position   = [1],
                        solve_rotation   = [1],
                        weight_position  = 1,
                        weight_rotation  = 1,
                        joi_ctrl_num= 6): 
                        
        ingredients = make_ik_input(target_name = target_name,
                                    target_pos  = target_position,
                                    target_rot  = target_rotation,
                                    solve_pos   = solve_position,
                                    solve_rot   = solve_rotation,
                                    weight_pos  = weight_position,
                                    weight_rot  = weight_rotation,
                                    joi_ctrl_num= joi_ctrl_num)
        total_q = self.chain.get_q_from_ik(ingredients)
        ctrl_q  = total_q[:joi_ctrl_num+1] #Including base joint for Unity Env
        ctrl_q  = ctrl_q.reshape(-1,)
        return ctrl_q 

### IK Solve: Panda

In [3]:
file_name = "../urdf/panda/panda.urdf"
base_offset = [0,0,0]
robot     = RobotClass(file_name=file_name, base_offset=base_offset)
q         = robot.solve_ik(target_name    = ['panda_joint7'],
            target_position  = [[0.,0,0.8]],
            target_rotation  = [[0, 0, 0]],
            solve_position   = [1],
            solve_rotation   = [1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 7)
print("Control joint value: {} \nShape: {}".format(q, q.shape))
for i in range(8):
    print(robot.chain.joint[i].name)
    print(robot.chain.joint[i].p)

Name: base_joint                         , ID: 1 , Mother_joint: 0 , Child_joint: [2]        , Type: fixed     , Axis: [[0. 0. 1.]]    , B_offset:[[0. 0. 0.]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: panda_joint1                       , ID: 2 , Mother_joint: 1 , Child_joint: [3]        , Type: revolute  , Axis: [[0. 0. 1.]]    , B_offset:[[0.    0.    0.333]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: panda_joint2                       , ID: 3 , Mother_joint: 2 , Child_joint: [4]        , Type: revolute  , Axis: [[0. 0. 1.]]    , B_offset:[[0. 0. 0.]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: panda_joint3                       , ID: 4 , Mother_joint: 3 , Child_joint: [5]        , Type: revolute  , Axis: [[0. 0. 1.]]    , B_offset:[[ 0.    -0.316  0.   ]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Nam

### IK Solve: UR5e

In [4]:
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0,0,0]
robot     = RobotClass(file_name=file_name, base_offset=base_offset)
q         = robot.solve_ik(target_name = ['wrist_3_joint'],
            target_position  = [[0.2,0,0.6]],
            target_rotation  = [[0, 0, 0]],
            solve_position   = [1],
            solve_rotation   = [1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 6)

print("Control joint value: {} \nShape: {}".format(q, q.shape))
for i in range(15):
    print(robot.chain.joint[i].name)
    print(robot.chain.joint[i].p)

Name: base_joint                         , ID: 1 , Mother_joint: 0 , Child_joint: [2]        , Type: fixed     , Axis: [[0. 0. 1.]]    , B_offset:[[0. 0. 0.]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: shoulder_pan_joint                 , ID: 2 , Mother_joint: 1 , Child_joint: [3]        , Type: revolute  , Axis: [[0. 0. 1.]]    , B_offset:[[0.    0.    0.163]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: shoulder_lift_joint                , ID: 3 , Mother_joint: 2 , Child_joint: [4]        , Type: revolute  , Axis: [[0. 1. 0.]]    , B_offset:[[0.    0.138 0.   ]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: elbow_joint                        , ID: 4 , Mother_joint: 3 , Child_joint: [5]        , Type: revolute  , Axis: [[0. 1. 0.]]    , B_offset:[[ 0.    -0.131  0.425]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 

In [5]:
import math 
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0,0,0]
joints      = np.array([0, -math.pi/2, math.pi/2, -math.pi/2, -math.pi/2, 0])
robot       = RobotClass(file_name=file_name, base_offset=base_offset)
# Index From Root 
idx_fr_root = find_route(robot.chain.joint, 7) # base joint + 6-DoF
print("Idx from root: {}".format(idx_fr_root))
add_joints(robot.chain.joint, idx_fr_root,joints)
robot.chain.fk_chain(1)
for i in range(8):
    print(robot.chain.joint[i].name)
    print(robot.chain.joint[i].p.T)


Name: base_joint                         , ID: 1 , Mother_joint: 0 , Child_joint: [2]        , Type: fixed     , Axis: [[0. 0. 1.]]    , B_offset:[[0. 0. 0.]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: shoulder_pan_joint                 , ID: 2 , Mother_joint: 1 , Child_joint: [3]        , Type: revolute  , Axis: [[0. 0. 1.]]    , B_offset:[[0.    0.    0.163]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: shoulder_lift_joint                , ID: 3 , Mother_joint: 2 , Child_joint: [4]        , Type: revolute  , Axis: [[0. 1. 0.]]    , B_offset:[[0.    0.138 0.   ]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: elbow_joint                        , ID: 4 , Mother_joint: 3 , Child_joint: [5]        , Type: revolute  , Axis: [[0. 1. 0.]]    , B_offset:[[ 0.    -0.131  0.425]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 

In [6]:
file_name = "../urdf/ur5e/ur5e_onrobot.urdf"
base_offset = [0,0,0]
robot     = RobotClass(file_name=file_name, base_offset=base_offset)
q         = robot.solve_ik(target_name = ['wrist_3_joint'],
            target_position  = [[0.492,0.134,0.588]],
            target_rotation  = [[-math.pi/2, 0, -math.pi/2]],
            solve_position   = [1],
            solve_rotation   = [1],
            weight_position  = 1,
            weight_rotation  = 1,
            joi_ctrl_num= 6)
print("Result: {}".format(q[1:])) # Without base joint  
print("Goal: {}".format(np.array([0, -math.pi/2, math.pi/2, -math.pi/2, -math.pi/2, 0])))

Name: base_joint                         , ID: 1 , Mother_joint: 0 , Child_joint: [2]        , Type: fixed     , Axis: [[0. 0. 1.]]    , B_offset:[[0. 0. 0.]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: shoulder_pan_joint                 , ID: 2 , Mother_joint: 1 , Child_joint: [3]        , Type: revolute  , Axis: [[0. 0. 1.]]    , B_offset:[[0.    0.    0.163]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: shoulder_lift_joint                , ID: 3 , Mother_joint: 2 , Child_joint: [4]        , Type: revolute  , Axis: [[0. 1. 0.]]    , B_offset:[[0.    0.138 0.   ]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Name: elbow_joint                        , ID: 4 , Mother_joint: 3 , Child_joint: [5]        , Type: revolute  , Axis: [[0. 1. 0.]]    , B_offset:[[ 0.    -0.131  0.425]]
Current Position: [[0 0 0]]
Current Rotation: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 